The goal of this notebook is to take the ad hoc implementation in `MIPExample1.ipynb` and vectorize the operations such that any size Tyche problem can be converted to a MIP and solved.

In [5]:
import os
import sys
sys.path.insert(0, os.path.abspath("../src"))

In [6]:
import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb
import tyche             as ty

from copy            import deepcopy
from IPython.display import Image 

In [7]:
import cProfile
import timeit

In [8]:
from mip import Model, MINIMIZE, MAXIMIZE, BINARY, xsum
from itertools import product, combinations

In [9]:
designs = ty.Designs("data")
investments = ty.Investments("data")
designs.compile()
tranche_results = investments.evaluate_tranches(designs, sample_count=250)
results = investments.tranches.join(tranche_results.summary)
evaluator = ty.Evaluator(investments.tranches, tranche_results.summary)

Get the wide-format interpolated elicitation data from the Tyche Evaluator and reset the multi-level index.

In [10]:
wide = evaluator.evaluate_corners_wide().reset_index()

In [30]:
wide.head(26)

Index,CIGS,CdTe,GaAs,InGaP,Perovskite,Polysilicon,Power Electronics,Soft Costs,Capital,Efficiency,GHG,Hazardous,LCOE,Lifetime,Strategic,Yield
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.593241,2.063635,-0.003592,0.975744,-0.143676,187.992761,0.063252,10015.086681
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,-1.508512,2.063835,-0.003592,0.975496,-0.141025,187.992712,0.063252,10015.083797
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,-1.328371,2.065188,-0.003592,0.974084,-0.135389,187.992747,0.063252,10015.085260
3,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,0.0,-1.516662,2.064005,-0.003592,0.963496,-0.137566,187.992684,0.063252,10079.299057
4,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,1000000.0,-1.431932,2.064204,-0.003592,0.963248,-0.134915,187.992636,0.063252,10079.296172
5,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,5000000.0,-1.251792,2.065558,-0.003592,0.961836,-0.129279,187.992670,0.063252,10079.297636
6,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,0.0,-1.437181,2.064132,-0.003592,0.956235,-0.133022,187.992734,0.063252,10121.321592
7,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,1000000.0,-1.352451,2.064332,-0.003592,0.955986,-0.130371,187.992686,0.063252,10121.318707
8,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,5000000.0,-1.172311,2.065686,-0.003592,0.954575,-0.124735,187.992720,0.063252,10121.320171
9,0.0,0.0,0.0,0.0,0.0,2500000.0,0.0,0.0,-1.474386,2.075568,-0.003592,0.933428,-0.138824,187.992737,0.063252,10034.358772


In [66]:
categories = ['Polysilicon', 'Power Electronics', 'Soft Costs']

metric_obj = ['Efficiency']

metric_constraint = ['Hazardous']

Pull out the investment values and metric values from the data set.

In [67]:
# Investment levels = interval endpoints
inv_levels = wide.loc[:26,categories].values.tolist()

# Elicited metric values for objective function
m = wide.loc[:26,metric_obj].values.tolist()

# Elicited metric values for constraint
const = wide.loc[:26, metric_constraint].values.tolist()

# Number of endpoints
I = len(inv_levels)

Instantiate the MIP optimization problem.

In [68]:
example = Model(sense=MAXIMIZE)

Create continuous $\lambda$ variables with lower bound 0.0 and upper bound 1.0

In [69]:
lmbd_vars = []

for i in range(I):
    lmbd_vars += [example.add_var(name='lmbd_' + str(i), lb=0.0, ub=1.0)]

Create binary (integer) variables $y_{in_i}$ and constraints on $y$ and $\lambda$ that restrict optimal solutions to the intervals between endpoints (elicited data).

In [70]:
bin_vars = []
bin_count = 0

for i in range(I):
    for j in range(i, I):
        if j != i:
            # create binary variable
            bin_vars += [example.add_var(name='y_' + str(i) + str(j),
                                         var_type=BINARY)]
            # add binary/lambda variable constraint
            example += bin_vars[bin_count] <= lmbd_vars[i] + lmbd_vars[j], 'Interval_Constraint_' + str(i) + str(j)
            bin_count += 1

Define upper bound for total budget constraint and per-category upper bounds.

In [71]:
B = 3000000.0
max_amount = [1000000.0, 2000000.0, 1500000.0]
const_amount = 0.96

Create total budget constraint as a function of the $\lambda$ variables and the elicited investment levels from the data set.

In [72]:
example += xsum(lmbd_vars[i] * inv_levels[i][j] 
                for i in range(I)
                for j in range(len(inv_levels[i]))) <= B, 'Total_Budget'

Create constraints for budget within each investment category.

In [73]:
for j in range(len(categories)):
    example += xsum(lmbd_vars[i] * [el[j] for el in inv_levels][i] 
                    for i in range(I)) <= max_amount[j], 'Budget_for_' + categories[j].replace(' ', '')

Create minimum-metric constraint on the lambda variables.

In [74]:
example += xsum(lmbd_vars[i] * const[i][0] for i in range(I)) <= const_amount

Create two sets of constraints:
* Convexity constraints on $\lambda$ variables: the $\lambda$s must sum to 1.
* Constrain binary $y$ variables within each category such that the decision variables lie within exactly one interval.

In [75]:
example += sum(lmbd_vars) == 1, 'Lambda_Sum'

The sum over all binary variables is constrained to be exactly 1 to restrict optimal solutions to the linear intervals between endpoints (elicited data).

In [76]:
example += sum(bin_vars) == 1, 'Binary_Sum'

Create objective function: Metric objective as a function of $\lambda$s and elicited data.

In [78]:
example.objective = xsum(m[i][0] * lmbd_vars[i] for i in range(I))

Save model to a `lp` file for manual examination (optional; un-comment to save).

In [79]:
#example.write('example.lp')

Optimize. Output of `<OptimizationStatus.OPTIMAL: 0>` indicates a feasible optimum was found.

In [80]:
example.optimize()

<OptimizationStatus.OPTIMAL: 0>

Print optimal objective function value.

In [81]:
example.objective_value

2.073294365394256

Get the optimal $\lambda$ and $y$ values. Store as separate lists and also print out.

In [62]:
lmbd_opt = []
y_opt = []

for v in example.vars:
    if 'lmbd' in v.name:
        lmbd_opt += [v.x]
    elif 'y' in v.name:
        y_opt += [v.x]
    print('{} : {}'.format(v.name, v.x))

lmbd_0 : 0.0
lmbd_1 : 0.0
lmbd_2 : 0.0
lmbd_3 : 0.0
lmbd_4 : 0.7777777777777778
lmbd_5 : 0.0
lmbd_6 : 0.0
lmbd_7 : 0.0
lmbd_8 : 0.0
lmbd_9 : 0.0
lmbd_10 : 0.0
lmbd_11 : 0.0
lmbd_12 : 0.0
lmbd_13 : 0.0
lmbd_14 : 0.0
lmbd_15 : 0.0
lmbd_16 : 0.0
lmbd_17 : 0.0
lmbd_18 : 0.0
lmbd_19 : 0.22222222222181964
lmbd_20 : 0.0
lmbd_21 : 0.0
lmbd_22 : 0.0
lmbd_23 : 0.0
lmbd_24 : 0.0
lmbd_25 : 0.0
lmbd_26 : 4.025622640582747e-13
y_01 : 0.0
y_02 : 0.0
y_03 : 0.0
y_04 : 0.0
y_05 : 0.0
y_06 : 0.0
y_07 : 0.0
y_08 : 0.0
y_09 : 0.0
y_010 : 0.0
y_011 : 0.0
y_012 : 0.0
y_013 : 0.0
y_014 : 0.0
y_015 : 0.0
y_016 : 0.0
y_017 : 0.0
y_018 : 0.0
y_019 : 0.0
y_020 : 0.0
y_021 : 0.0
y_022 : 0.0
y_023 : 0.0
y_024 : 0.0
y_025 : 0.0
y_026 : 0.0
y_12 : 0.0
y_13 : 0.0
y_14 : 0.0
y_15 : 0.0
y_16 : 0.0
y_17 : 0.0
y_18 : 0.0
y_19 : 0.0
y_110 : 0.0
y_111 : 0.0
y_112 : 0.0
y_113 : 0.0
y_114 : 0.0
y_115 : 0.0
y_116 : 0.0
y_117 : 0.0
y_118 : 0.0
y_119 : 0.0
y_120 : 0.0
y_121 : 0.0
y_122 : 0.0
y_123 : 0.0
y_124 : 0.0
y_125 : 0.0


Calculate the optimal investment levels.

In [82]:
inv_levels_opt = {}

for i in range(len(categories)):
    inv_levels_opt[categories[i]] = sum([lmbd_opt[j] * [el[i] for el in inv_levels][j] 
                                         for j in range(len(lmbd_opt))])

inv_levels_opt

{'Polysilicon': 999999.9999999999,
 'Power Electronics': 777777.7777797906,
 'Soft Costs': 1000000.0000016103}

Calculate the optimal total investment.

In [83]:
total_inv_levels_opt = sum(inv_levels_opt.values())
total_inv_levels_opt

2777777.777781401